In [8]:
import pandas as pd 
import numpy as np
import rasterio as rio
import rasterio.features
from rasterio.warp import reproject
import geopandas as gpd
from shapely.wkt import loads
from shapely.geometry import Point
import matplotlib as mpl  
from matplotlib import pyplot as plt
from geocube.api.core import make_geocube
from osgeo import gdal

In [22]:
edges = pd.read_csv('../network_inputs/bolinas_edges_sim.csv')
edges = gpd.GeoDataFrame(edges, crs='epsg:4326', geometry=edges['geometry'].map(loads))
edges['start_x'] = edges['geometry'].apply(lambda x: x.coords[0][0])
edges['end_x'] = edges['geometry'].apply(lambda x: x.coords[-1][0])

edges1 = edges.loc[edges['start_x']<=edges['end_x']]
cube = make_geocube(vector_data=edges1, measurements=["eid"], resolution=(-0.0002, 0.0002))
cube["eid"].rio.to_raster("../network_inputs/bolinas_edges_raster1.tif")

edges2 = edges.loc[edges['start_x']>edges['end_x']]
cube = make_geocube(vector_data=edges2, measurements=["eid"], resolution=(-0.0002, 0.0002))
cube["eid"].rio.to_raster("../network_inputs/bolinas_edges_raster2.tif")

In [43]:
# resample fire to match road
# open reference file and get resolution
referenceFile = "../network_inputs/bolinas_edges_raster1.tif"
reference = gdal.Open(referenceFile, 0)  # this opens the file in only reading mode
referenceTrans = reference.GetGeoTransform()
print(referenceTrans)
x_min = referenceTrans[0]
y_max = referenceTrans[3]
x_res = referenceTrans[1]
y_res = -referenceTrans[5]  # make sure this value is positive
x_max = x_min + x_res*reference.RasterXSize
y_min = y_max - y_res*reference.RasterYSize
ref_proj = reference.GetProjection()
print(x_min, y_min, x_max, y_max, x_res, y_res)


for fire_id in [1,2,3]:
    for hour_id in range(1,11):
        # specify input and output filenames
        inputFile = "flamelength/flame_length_scen{}/{}.TIF".format(fire_id, hour_id)
        outputFile = "flamelength/flame_length_resampled_raster/flame_length_f{}_h{}.tif".format(fire_id, hour_id)
        dataset = gdal.Open(inputFile)
        # print(dataset)
        # print(dataset.GetProjection())

        # call gdal Warp
        kwargs = {"format": "GTiff", "xRes": x_res, "yRes": y_res, "dstSRS": ref_proj}#, "outputBounds": [ x_min, y_min, x_max, y_max]}
        ds = gdal.Warp(outputFile, dataset, outputBounds=[ x_min, y_min, x_max, y_max], **kwargs)
        # print(ds.GetGeoTransform(), ds.RasterXSize, ds.RasterYSize)

(-122.74640000000001, 0.00019999999999998825, 0.0, 37.9804, 0.0, -0.0001999999999999982)
-122.74640000000001 37.879200000000004 -122.59920000000001 37.9804 0.00019999999999998825 0.0001999999999999982
